Created by: Iyare Oseghae | Date: 6/11/2025

Last Updated: June 11, 2025

The purpose of this code is to simplify and streamline the task of converting NASA Pandora Data from Text file to dataframe and excel spreadsheet format. Pandora data can be found at the Pandonio Global Network Data Portal: https://data.pandonia-global-network.org/. Link to Git Repository with other helpul tools for working with Pandora Data: https://github.com/Iyare-O/Pandora-Unboxed.

# Imports

In [1]:
import pandas as pd
import re
import os
import numpy as np

# Making The Data Usable: Converting from .txt to . xlsx

This code block converts the pandora data from txt format to a Pandas dataFrame and saves the dataFrame as a .xlsx file

In [7]:
file_path = "Insert_Path/PandoraXXs1_CityST_L2_rnvh3p1-8.txt" #(NO2)

# Read the file
with open(file_path, 'r') as file:
    lines = file.readlines()

# Extract column names and descriptions
columns = []
data_start_idx = 0
for i, line in enumerate(lines):
    if re.match(r"^Column \d+:", line):
        column_name = line.split(":")[1].strip()
        columns.append(column_name)
    elif re.match(r"^-+$", line):
        data_start_idx = i + 1
        break

# Extract data lines and split into lists
data_lines = lines[data_start_idx:]

# find column names in the .txt file
columns = [item for item in data_lines if item.startswith("Column")]

#remove "/n from column header names"
columns = [col.strip() for col in columns]

# Shorten column names to first 9 characters
short_columns = [col[:9] for col in columns]

# Define the regular expression pattern for the desired format
pattern = r'^(\d{4})(\d{2})(\d{2})T(\d{2})(\d{2})(\d{2})\.\dZ'

# Extract lines that match the pattern
matching_lines = [line for line in data_lines if re.match(pattern, line)]

# Extract year, month, day, and time (hhmmss) from the Timestamp column
extracted_data = [re.match(pattern, line).groups() if re.match(pattern, line) else (None, None, None, None, None, None) for line in matching_lines]

# Split each line into elements and limit to the length of columns
split_lines = [line.split()[:len(columns)] for line in matching_lines]

# Convert the list of lists into a pandas DataFrame
df = pd.DataFrame(split_lines, columns=short_columns[:len(columns)])

# Insert new columns for year, month, day, and time (hhmmss)
df.insert(0, 'Year', [data[0] for data in extracted_data])
df.insert(1, 'Month', [data[1] for data in extracted_data])
df.insert(2, 'Day', [data[2] for data in extracted_data])
df.insert(3, 'Time', ['{}:{}:{}'.format(data[3], data[4], data[5]) for data in extracted_data])


df['Year'] = df['Year'].astype(int)
df['Month'] = df['Month'].astype(int)
df['Day'] = df['Day'].astype(int)

df.to_excel("Save_Path/insert_file_name.xlsx", index=False)
print(df.head())

   Year  Month  Day      Time           Column 1:    Column 2: Column 3:  \
0  2025      1    6  18:54:02  20250106T185402.4Z  9137.787528     27.01   
1  2025      1    6  19:04:26  20250106T190426.3Z  9137.794749     26.96   
2  2025      1    6  19:14:47  20250106T191447.5Z  9137.801938     26.95   
3  2025      1    6  19:25:08  20250106T192508.8Z  9137.809130     26.98   
4  2025      1    6  19:38:06  20250106T193806.2Z  9137.818127     27.01   

  Column 4: Column 5: Column 6:  ...  Column 60  Column 61   Column 62  \
0     51.92    184.09     79.73  ...  3.590e-05  4.389e-06  5.3166e-05   
1     52.14    187.13     77.51  ...  3.624e-05  4.389e-06  4.8987e-05   
2     52.48    190.12     75.29  ...  3.657e-05  4.389e-06  5.0770e-05   
3     52.93    193.07     73.08  ...  3.690e-05  4.389e-06  4.9654e-05   
4     53.66    196.70     70.31  ...  3.731e-05  4.389e-06  4.1852e-05   

    Column 63 Column 64 Column 65 Column 66   Column 67 Column 68   Column 69  
0  1.3234e-06     

array([1, 2, 3, 4, 5, 6])

# Highlight The Missing Data

It is not uncommon that Pandora will have some data missing, get ahead of the game and find out which days are missing.

In [38]:
import pandas as pd
from datetime import datetime

# === Load your preprocessed data (already has Year, Month, Day) ===
# If you've already got sa_df from the previous script, you can skip the Excel read
file_path = "Insert_file.xlsx"
df = pd.read_excel(file_path)

# === Create a proper datetime column ===
sa_df['Date'] = pd.to_datetime(sa_df[['Year', 'Month', 'Day']])

# === Generate full date range ===
min_date = sa_df['Date'].min()
max_date = sa_df['Date'].max()
all_dates = pd.date_range(start=min_date, end=max_date, freq='D')

# === Get all available unique dates from the dataset ===
available_dates = pd.to_datetime(sa_df['Date'].unique())

# === Find missing dates ===
missing_dates = all_dates.difference(available_dates)

# === Output missing days ===
missing_df = pd.DataFrame({'MissingDate': missing_dates})
print("Days with NO DATA available:")
print(missing_df)

# === Optional: save to Excel ===
output_path = "file_path/missing_dates.xlsx"
missing_df.to_excel(output_path, index=False)
print(f"Missing dates exported to: {output_path}")

🚫 Days with NO DATA available:
   MissingDate
0   2025-03-07
1   2025-03-08
2   2025-03-09
3   2025-03-10
4   2025-03-11
5   2025-03-12
6   2025-03-13
7   2025-04-07
8   2025-04-08
9   2025-04-09
10  2025-04-24
11  2025-05-05
12  2025-05-06
13  2025-05-07
14  2025-05-08
15  2025-05-09
16  2025-05-10
17  2025-05-11
18  2025-05-12
19  2025-05-13
20  2025-05-14
21  2025-05-15
22  2025-05-16
23  2025-05-17
24  2025-05-18
25  2025-05-19
26  2025-05-20
27  2025-05-21
 Missing dates exported to: File_Name

# Locating you Varibales of Interest

This code block print a list of the column numbers and their corresponding variable. This code block is very important since it helps us know exatly where to look for your variable(s) of interest.

In [24]:
list(columns)

['Column 1: UT date and time for measurement center, yyyymmddThhmmssZ (ISO 8601)',
 'Column 2: Fractional days since 1-Jan-2000 UT midnight for measurement center',
 'Column 3: Effective duration of measurement [s]',
 'Column 4: Solar zenith angle for measurement center [deg]',
 'Column 5: Solar azimuth for measurement center [deg], 0=north, increases clockwise',
 'Column 6: Lunar zenith angle for measurement center [deg]',
 'Column 7: Lunar azimuth for measurement center [deg], 0=north, increases clockwise',
 'Column 8: Pointing zenith angle for measurement center [deg]',
 'Column 9: Pointing azimuth for measurement center [deg], 0=north, increases clockwise',
 'Column 10: rms of unweighted fitting residuals, -9=fitting not successful',
 'Column 11: Normalized rms of fitting residuals weighted with independent uncertainty, -9=fitting not successful or no uncertainty given',
 'Column 12: Expected rms of unweighted fitting residuals based on independent uncertainty, -9=fitting not succe

I would suggest to make note the variables of interest once you have located them, see below.

Column 56: Nitrogen dioxide surface concentration [mol/m3]
    
Column 62: Nitrogen dioxide tropospheric vertical column amount [moles per square meter]

# Next See the "Filtering and Plotting" code file to find out how to filter your Pandora data for specific time frames and how to plot the data to your liking.